In [17]:
def searchSubStrInList(mylist, sub="insrt_ts "):
    return [s for s in mylist if sub in s]

#xlfile = "Portal prfl schema de-ID requirements Final Approved 4-16-2024.xlsx"
xlfile = "Portal ptl schema de-ID requirements final approved 4-16-2024.xlsx"
#xlfile = "Portal audt schema de-ID requirements APPROVED 4-9-2024.xlsx"
stname = "Sheet1"

#schsql = 'ocse-syt-main-comm-table-list.sql'
#schsql = 'ocse-syt-main-ofcr-table-list.sql'
#schsql = 'ocse-syt-main-prfl-table-list.sql'
schsql = 'ocse-syt-main-ptl-table-list.sql'
#schsql = 'ocse-syt-main-audt-table-list.sql'

tabdict = {}
cnt = 0
tabCols = []
startTab = False
tabkey = "Empty"

file1 = open(schsql, 'r')
for line in file1:
    cnt += 1
    
    if not startTab:
        tabCols = []
    if "CREATE TABLE" in line.strip():
        tabkey = line.split(" ")[2]
        startTab = True
        continue

    if "CONSTRAINT" in line.strip():
        if startTab:
            tabdict[tabkey] = tabCols
        startTab = False
        tabCols = []
        tabkey = "Empty"        
        
    if startTab:
        tabCols.append(line.strip())        
        
        
    #print("{}: {}  {}  {}  {}".format(cnt, line.strip(), tabkey, tabCols, startTab))
file1.close()


if "Empty" in tabdict.keys():
    print("There is some issue -- ", tabdict.keys() )
    raise SystemExit("Reading Tables issue")

for ll in tabdict.values():
    if len(ll)==0:
        raise SystemExit("Reading Cols issue")        

print("Table List::  ", tabdict.keys())
print("Total Tables::  ", len(tabdict.keys()))


for kk in tabdict.keys():
    if ('ofcr' in schsql) or ('ptl' in schsql):
        tabdict[kk].append("dms_commit_ts timestamp NOT NULL")


#for kk in tabdict.keys():
#    if len(  searchSubStrInList( tabdict[kk] ) )==0:
#        tabdict[kk].append("dms_commit_ts timestamp NOT NULL")

Table List::   dict_keys(['ptl.account_locations', 'ptl.account_status_ref', 'ptl.action_typ_ref', 'ptl.addr_typ_ref', 'ptl.alternative_org_address', 'ptl.alternative_org_address_typ', 'ptl.app_clsfn_ref', 'ptl.contact', 'ptl.contact_typ_ref', 'ptl.country_ref', 'ptl.empr_certn_elgt_stus_ref', 'ptl.org_prfl', 'ptl.org_prfl_addr_typ', 'ptl.org_prfl_vrfn_cd_ref', 'ptl.org_relationship', 'ptl.org_relationship_typ_ref', 'ptl.org_resplt', 'ptl.org_resplt_ref', 'ptl.org_role_app_relationship', 'ptl.org_role_typ_ref', 'ptl.org_typ_ref_org_resplt_ref_xref', 'ptl.org_typ_ref_org_role_typ_ref_xref', 'ptl.page_typ_ref', 'ptl.prfl_role_app_relationship', 'ptl.role_app_relationship', 'ptl.role_page_relationship', 'ptl.role_typ_ref', 'ptl.state_county_ref', 'ptl.state_role_app_relationship', 'ptl.user_account', 'ptl.user_affiln_ref', 'ptl.user_affiln_subfn_ref', 'ptl.user_afiln_subfn_role_typ_xref', 'ptl.user_org_relationship', 'ptl.user_role_app_relationship'])
Total Tables::   35


In [18]:
import json
import pandas as pd
import numpy as np


dbName = ""
alldb = tabdict.keys()
alldb = [x.split(".")[0] for x in alldb]
alldb = list( set(alldb) )
if len(alldb)>1:
    print("more than one DB::  ", alldb)
    raise SystemExit("This file have more than One Schema")  
    
dbName = alldb[0].capitalize()
print("Working for Schema::  ", dbName)

Working for Schema::   Ptl


In [19]:
initialGlue = """AWSTemplateFormatVersion: '2010-09-09'
Description: 'Creates OCSE environment Data Analytics Glue databases and tables.'
# Data type change rules for Glue tables:
# NUMBER: if > 10, bigint, else int
# SMALLINT: do not change
# DECIMAL: do not change
# TIMESTAMP: do not change
# DATE: do not change if YYYY-MM-DD format, else string 
# the rest: string  # includes "character varying", character, varchar2
Parameters:
  EnvId:
    Description: 'Environment ID'
    Type: String
    AllowedValues:
      - sandbox
      - syt
      - val
      - prd
Mappings:
  S3ConfigByEnvId:
    sandbox:
      DeidMainRawGlueDbS3Location: 's3://ocse-sandbox-deid-main-raw-data/glue-db/'
    syt:
      DeidMainRawGlueDbS3Location: 's3://ocse-syt-deid-main-raw-data/glue-db/'
      DeidMainRawGlueDatabase: 'ocse-syt-deid-main-raw-glue-db'
    val:
      DeidMainRawGlueDbS3Location: 's3://ocse-val-deid-main-raw-data/glue-db/'
      DeidMainRawGlueDatabase: 'ocse-val-deid-main-raw-glue-db'
    prd:
      DeidMainRawGlueDbS3Location: 's3://ocse-prd-deid-main-raw-data/glue-db/'
      DeidMainRawGlueDatabase: 'ocse-prd-deid-main-raw-glue-db'
Resources:
"""

allgluetab = []

for kk, vv in tabdict.items():
    aa = kk.replace(".", "/")
    bb = kk.replace(".", "_")
    dd = "".join(x.capitalize() for x in bb.split('_'))
    #print(aa, bb, dd, kk, vv)
        
    gluetab = f"""  DeidMainRaw{dbName}{dd}UserGlueTable:
    Type: AWS::Glue::Table
    Properties:
      CatalogId: !Ref AWS::AccountId
      DatabaseName: !FindInMap [S3ConfigByEnvId, !Ref EnvId, DeidMainRawGlueDatabase]
      TableInput:
        Name: {bb}
        Description: Deid Main Raw {aa} Glue Table
        TableType: EXTERNAL_TABLE
        Parameters: {{
          "EXTERNAL": "TRUE", 
          "classification": "parquet",
          "columnsOrdered": "true",
          "compressionType": "none",
          "typeOfData": "file"
        }}
        PartitionKeys: []
        StorageDescriptor:
          Location: !Join ['', [!FindInMap [S3ConfigByEnvId, !Ref EnvId, DeidMainRawGlueDbS3Location], '{aa}/']]
          StoredAsSubDirectories: False
          InputFormat: org.apache.hadoop.mapred.TextInputFormat
          OutputFormat: org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat
          SerdeInfo:
            SerializationLibrary: org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe
          Columns:
"""

    allcols = [f"""            - Name: op
              Type: string
"""           ]
    for col in vv:
        print(col, col.split(" "))
        coln = col.split(" ")[0]
        typn = col.split(" ")[1]
        typm = "string"
        if ("bigserial" in typn) or ("int" in typn):
            typm = "int"
        if "timestamp" in typn:
            typm = "timestamp"
        if "date" in typn:
            typm = "date"            
        print(col, coln, typm)
        nextcol = f"""            - Name: {coln}
              Type: {typm}
"""
        allcols.append(nextcol)

    #print("".join(x for x in allcols))
    gluetab = gluetab + "".join(x for x in allcols)
    allgluetab.append(gluetab)

outgluefile = "YAMLfiles/ocse-syt-deid-main-"+str(dbName.lower())+"-schema-glue.yaml"
with open(outgluefile, 'w') as f:
    f.write(f"{initialGlue}")   
    for line in allgluetab:
        f.write(f"{line}")        

org_prfl_uid int8 NOT NULL, ['org_prfl_uid', 'int8', 'NOT', 'NULL,']
org_prfl_uid int8 NOT NULL, org_prfl_uid int
state_cd bpchar(2) NOT NULL, ['state_cd', 'bpchar(2)', 'NOT', 'NULL,']
state_cd bpchar(2) NOT NULL, state_cd string
dms_commit_ts timestamp NOT NULL ['dms_commit_ts', 'timestamp', 'NOT', 'NULL']
dms_commit_ts timestamp NOT NULL dms_commit_ts timestamp
account_status_cd bpchar(1) NOT NULL, ['account_status_cd', 'bpchar(1)', 'NOT', 'NULL,']
account_status_cd bpchar(1) NOT NULL, account_status_cd string
account_status_desc_txt varchar(50) NOT NULL, ['account_status_desc_txt', 'varchar(50)', 'NOT', 'NULL,']
account_status_desc_txt varchar(50) NOT NULL, account_status_desc_txt string
dms_commit_ts timestamp NOT NULL ['dms_commit_ts', 'timestamp', 'NOT', 'NULL']
dms_commit_ts timestamp NOT NULL dms_commit_ts timestamp
action_typ_cd bpchar(1) NOT NULL, ['action_typ_cd', 'bpchar(1)', 'NOT', 'NULL,']
action_typ_cd bpchar(1) NOT NULL, action_typ_cd string
action_typ_desc_txt varchar(

In [20]:
initialCrwl = """AWSTemplateFormatVersion: '2010-09-09'
Description: 'Creates OCSE environment Data Analytics Glue databases and tables.'
# Data type change rules for Glue tables:
# NUMBER: if > 10, bigint, else int
# SMALLINT: do not change
# DECIMAL: do not change
# TIMESTAMP: do not change
# DATE: do not change if YYYY-MM-DD format, else string 
# the rest: string  # includes "character varying", character, varchar2
Parameters:
  EnvId:
    Description: 'Environment ID'
    Type: String
    AllowedValues:
      - sandbox
      - syt
      - val
      - prd
Mappings:
  S3ConfigByEnvId:
    sandbox:
      CuratedGlueDbS3Location: 's3://ocse-sandbox-da-main-curated-data/glue-db/'
      DatabaseName: ocse-sandbox-da-main-curated-glue-db
    syt:
      CuratedGlueDbS3Location: 's3://ocse-syt-da-main-curated-data/glue-db/'
      DatabaseName: ocse-syt-da-main-curated-glue-db
    val:
      CuratedGlueDbS3Location: 's3://ocse-val-da-main-curated-data/glue-db/'
      DatabaseName: ocse-val-da-main-curated-glue-db
    prd:
      CuratedGlueDbS3Location: 's3://ocse-prd-da-main-curated-data/glue-db/'
      DatabaseName: ocse-prd-da-main-curated-glue-db
Resources:"""

allcrawler = []

for kk, vv in tabdict.items():
    aa = kk.replace(".", "/")
    bb = kk.replace(".", "_")
    dd = "".join(x.capitalize() for x in bb.split('_'))
    ee = aa.replace("/", "/"+dbName.lower()+"_")
    #print(aa, bb, dd, ee)
    crawler = f"""
  Curated{dd}Crawler:
    Type: AWS::Glue::Crawler
    Properties:
      Name: !Sub ocse-da-main-{bb}-curated-crawler
      DatabaseName: !FindInMap [S3ConfigByEnvId, !Ref EnvId, DatabaseName]
      Description: Join ['', [!Ref EnvId, 'Crawler on Curated database']]
      RecrawlPolicy: 
          RecrawlBehavior: CRAWL_EVERYTHING
      Role: !Sub 'arn:aws:iam::${{AWS::AccountId}}:role/OcseDaMainGlueServiceRole'
      SchemaChangePolicy: 
          UpdateBehavior: UPDATE_IN_DATABASE
          DeleteBehavior: DELETE_FROM_DATABASE            
      Targets:
        DeltaTargets: 
          - CreateNativeDeltaTable: true
            DeltaTables: 
            - !Join ['', [!FindInMap [S3ConfigByEnvId, !Ref EnvId, CuratedGlueDbS3Location], '{ee}/']]
            WriteManifest: false"""
    allcrawler.append(crawler)
    
outcrwlfile = "YAMLfiles/ocse-syt-da-main-"+str(dbName.lower())+"-schema-crawler.yaml"
with open(outcrwlfile, 'w') as f:
    f.write(f"{initialCrwl}") 
    for line in allcrawler:
        f.write(f"{line}")        

In [21]:
df = pd.read_excel(open(xlfile, 'rb'), sheet_name=stname, skiprows=[0])
#print(df.columns)
selcols = ['Schema', 'Table', 'Column', 'Final De-ID Requirement', ]
df = df[selcols]
#print(df.columns)
print(df['Final De-ID Requirement'].unique())

knownList = [np.nan ,'None Required', 'Hash', 'Exclude']
compList = df['Final De-ID Requirement'].unique()
diffList1 = list(set(knownList) - set(compList))
diffList2 = list(set(compList) - set(knownList))
print("*******  Difference  ************")
print(diffList1)
print(diffList2)
print("********************************")

[nan 'None Required' 'Hash' 'Exclude']
*******  Difference  ************
[]
[]
********************************


C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [22]:
pkeydict = {}
file1 = open(schsql, 'r')
for line in file1:
    cnt += 1
    if "CREATE TABLE" in line.strip():
        tabkey = line.split(" ")[2]
    if "PRIMARY KEY" in line.strip():
        pkeydict[tabkey] = line.strip().split("PRIMARY KEY")[-1].split(" ")
file1.close()

def cleanPK(vv):
    nvv = []
    for i in vv:
        if i=="":
            continue
        i = i.replace("(", "")
        i = i.replace(")", "")
        i = i.replace(",", "")
        nvv.append(i)
    return nvv

for kk in pkeydict.keys():
    pkeydict[kk] = cleanPK( pkeydict[kk] )
    
#print(pkeydict)

In [23]:
#for kk in pkeydict.keys():
#    sch = kk.split(".")[0]
#    tab = kk.split(".")[1]
#    cols = pkeydict[kk]
#    print(sch, tab, cols)


def onfilterDF(df, sch, tab):
    df = df[ (df['Schema']==sch) & (df['Table']==tab) ]
    excols = df[df['Final De-ID Requirement'].str.lower()=="exclude"]['Column'].tolist()
    hscols = df[df['Final De-ID Requirement'].str.lower()=="hash"]['Column'].tolist()
    excols = json.dumps(excols)
    #print(df)
    #print("excols: ", excols)
    #print("hscols: ", tab, hscols)
    hsdict = {}
    for hs in hscols:
        if "email" in hs:
            hsdict[hs] = "email"
        if "ssn" in hs:
            hsdict[hs] = "ssn"
        if "_id" in hs:
            hsdict[hs] = "case_id"
        if "fein" in hs:
            hsdict[hs] = "fein"
        if ("fnm" in hs) or ("mnm" in hs) or ("lnm" in hs):
            hsdict[hs] = "person_nm"  
        if ("phone" in hs) or ("phnnum_txt" in hs):
            hsdict[hs] = "phone_num"              
    hsdict = json.dumps(hsdict)   
    return (excols, hsdict)
    
rawToClnJsn = []
rawToCurJsn = []    
    
#for kk in list ( pkeydict.keys() )[:2]:
for kk in pkeydict.keys():
    sch = kk.split(".")[0]
    tab = kk.split(".")[1]
    cols = pkeydict[kk]
    excols, hsdict = onfilterDF(df, sch, tab)
    excols = str(excols)
    hsdict = str(hsdict)
    pklist = str(  json.dumps(pkeydict[kk])  )
    dtc = str({})
    tst = str({})
    rnc = str({})
    patlist = str([])
    s3tagpath = sch+ "/" + dbName.lower()+"_"+tab
    tabname = kk.replace(".", "_")
    #print(kk, pklist, excols, hsdict)
    emptyList = str([])
    emptyDict = str({})
    
    clnjsn = f""",
        "{tabname}": {{
            "table_name": "{tabname}",
            "primary_key_col": {pklist},
            "partition_cols": {patlist},
            "pii_col": {hsdict},
            "exclude_cols": {excols},
            "s3_target_write_folder": "{s3tagpath}",
            "date_trns_cols": {dtc},
            "ts_trns_cols": {tst},
            "rename_cols": {rnc}
        }}"""
    #print(clnjsn)
    rawToClnJsn.append(clnjsn)

    curjsn = f""",
        "{tabname}": {{
            "table_name": "{tabname}",
            "primary_key_col": {pklist},
            "partition_cols": {emptyList},
            "pii_col": {emptyDict},
            "exclude_cols": {emptyList},
            "s3_target_write_folder": "{s3tagpath}",
            "date_trns_cols": {emptyDict},
            "ts_trns_cols": {emptyDict},
            "rename_cols": {emptyDict}
        }}"""    
    rawToCurJsn.append(curjsn)    
    
    
outjsfile = "JSONfiles/glue-job-json-"+str(dbName.lower())+"-raw-clean.json"
with open(outjsfile, 'w') as f:
    for line in rawToClnJsn:
        f.write(f"{line}")
        
outjsfile = "JSONfiles/glue-job-json-"+str(dbName.lower())+"-clean-curated.json"
with open(outjsfile, 'w') as f:
    for line in rawToCurJsn:
        f.write(f"{line}")
        
print("Done:: ", outjsfile)

Done::  JSONfiles/glue-job-json-ptl-clean-curated.json


In [24]:
for kk in pkeydict.keys():
#    sch = kk.split(".")[0]
    tab = kk.split(".")[1].rstrip()
#    cols = pkeydict[kk]
#    print(sch, tab, cols)
    print("\"",str(tab).replace(" ", ""),"\"",",")

" account_locations " ,
" account_status_ref " ,
" action_typ_ref " ,
" addr_typ_ref " ,
" alternative_org_address " ,
" alternative_org_address_typ " ,
" app_clsfn_ref " ,
" contact " ,
" contact_typ_ref " ,
" country_ref " ,
" empr_certn_elgt_stus_ref " ,
" org_prfl " ,
" org_prfl_addr_typ " ,
" org_prfl_vrfn_cd_ref " ,
" org_relationship " ,
" org_relationship_typ_ref " ,
" org_resplt " ,
" org_resplt_ref " ,
" org_role_app_relationship " ,
" org_role_typ_ref " ,
" org_typ_ref_org_resplt_ref_xref " ,
" org_typ_ref_org_role_typ_ref_xref " ,
" page_typ_ref " ,
" prfl_role_app_relationship " ,
" role_app_relationship " ,
" role_page_relationship " ,
" role_typ_ref " ,
" state_county_ref " ,
" state_role_app_relationship " ,
" user_account " ,
" user_affiln_ref " ,
" user_affiln_subfn_ref " ,
" user_afiln_subfn_role_typ_xref " ,
" user_org_relationship " ,
" user_role_app_relationship " ,
